# Image Analysis with Python - <font color='teal'>Tutorial Pipeline Section 3</font>

*originally created in 2016*<br>
*updated and converted to a Jupyter notebook in 2017*<br>
*updated and converted to python 3 in 2018*<br>
*by Jonas Hartmann (Gilmour group, EMBL Heidelberg)*<br>
*updated in 2022 by Cheng-Yu Huang*<br>

## \**BONUS\** >> Batch Processing <a id=batch></a>

#### Background

In practice, we never work with just a single image, so we would like to make it possible to run our analysis pipeline for multiple images and then collect and analyze all the results. This final section of the tutorial shows how to do just that.

#### <font color='teal'>Exercise</font>

To run a pipeline multiple times, it needs to be packaged into a function or - even better - as a separate module. Jupyter notebook is not well suited for this, so if you're working in a notebook, first extract your code to a `.py` file (see instructions below). If you are not working in a notebook, create a copy of your pipeline; we will modify this copy into a function that can then be called repeatedly for different images.

To export a jupyter notebook as a `.py` file, use `File > Download as > Python (.py)`, then save the file. Open the resulting python script in a text editor or in an IDE like PyCharm. 


#### Let's clean the script a bit:

- Remove the line `%matplotlib [inline|notebook|qt]`. It is not valid python code outside of a Jupyter notebook.


- Go through the script and comment out everything related to plotting; when running a pipeline for dozens or hundreds of images, we usually do not want to generate tons of plots. Similarly, it can make sense to remove some print statments if you have many of them.


- Remove the sections `Manual Thresholding` and `Connected Components Labeling`; they are not used in the final segmentation.


- Remove the sections `Simple Analysis and Visualization` and `Writing Output to Files`; we will collect the output for each image when running the pipeline in a loop. That way, everything can be analyzed at once at the end. 
    - Note that, even though we skip it here, it is often very useful to store every input file's corresponding outputs in new files. When doing so, the output files should use the name of the input file modified with an additional suffix. For example, the results extracted when analyzing `img_1.tif` might best be stored as `img_1_results.pkl`.
    - You can implement this approach for saving the segmentations and/or the result dicts as a *bonus* exercise!


- Feel free to delete some of the background information to make the script more concise.


#### Converting the pipeline to a function:

Convert the entire pipeline into a function that accepts a directory and a filename as input, runs everything, and returns the final segmentation and the results dictionary. To do this, you must:

- Add the function definition statement at the beginning of the script (after the imports)
- Replace the 'hard-coded' directory path and filename by variables that are accepted by the function
- Indent all the code
- Add a return statement at the end


#### Importing the function and running it for multiple input files:

To actually run the pipeline function for multiple input files, we need to do the following:

- Import the pipeline function from the `.py` file
- Iterate over all the filenames in a directory
- For each filename, call the pipeline function
- Collect the returned results

Once you have converted your pipeline into a function as described above, you can import and run it according to the instructions below.

In [ ]:
###

#### Solution Note

The converted version of the pipeline can be found in `batch_processing_solution.py`. 

Note that the most of the exercise comments have been removed so it doesn't look too cluttered. However, this level of clean-up is probably a bit extreme; it is generally recommended to retain at least basic comments on the purpose of each code block. 

Also note that a doc string was added to the function definition (the string designated with three `"""` at the start and end, directly under the function definition). This is a very useful reference, since it is automatically recognized as a help message by Jupyter notebook (and other IDEs), so you can easily double-check what an imported function does, for example by typing `run_pipeline?` in a code cell.

In [ ]:
# (i) Test if your pipeline function actually works

# Import your function using the normal python syntax for imports, like this:
#   from your_module import your_function
# Run the function and visualize the resulting segmentation. Make sure everything works as intended.

from batch_processing_solution import run_pipeline
pip_seg, pip_results = run_pipeline(r"example_data", r'example_cells_1.tif')

plt.figure(figsize=(7,7))
plt.imshow(np.zeros_like(pip_seg), interpolation='none', cmap='gray', vmax=1)  # Simple black background
plt.imshow(np.ma.array(pip_seg, mask=pip_seg==0), interpolation='none', cmap='prism')
plt.show()

In [ ]:

# (ii) Get all relevant filenames from the input directory

# Use the function 'listdir' from the module 'os' to get a list of all the files
# in a directory. Find a way to filter out only the relevant input files, namely
# "example_cells_1.tif" and "example_cells_2.tif". Of course, one would usually
# do this for many more images, otherwise it's not worth the effort.
# Hint: Loop over the filenames and use if statements to decide which ones to 
#       keep and which ones to throw away.

# Get all files
dirpath = r"example_data"
from os import listdir
filelist = listdir(dirpath)

# Filter for target files: simple option
# Note that this will use ALL files with a .tif ending, which in some circumstances
# may include files that are not supposed to be used!
target_files = []
for fname in filelist:
    if fname.endswith('.tif'):
        target_files.append(fname)
print(target_files)

# Filter for target files: advanced option using regex and a list comprehension
import re
target_pattern = re.compile("^example_cells_\d+\.tif$")
target_files = [fname for fname in filelist if target_pattern.match(fname)]
print(target_files)

In [ ]:
# (iii) Iterate over the input filenames and run the pipeline function

# Be sure to collect the output of the pipeline function in a way that allows
# you to trace it back to the file it came from. You could for example use a
# dictionary with the filenames as keys.

# Initialize empty dictionaries
all_seg = {}
all_results = {}

# Iterate over files and run pipeline for each
for fname in target_files:
    pip_seg, pip_results = run_pipeline(dirpath, fname)
    all_seg[fname] = pip_seg
    all_results[fname] = pip_results

In [ ]:
# (iv) Recreate one of the scatterplots from above but this time with all the cells

# You can color-code the dots to indicate which file they came from. Don't forget to
# add a corresponding legend.

plt.figure(figsize=(8,5))

colors = ['blue','red']
for key, color in zip(sorted(all_results.keys()), colors):
    plt.scatter(all_results[key]["cell_area"], all_results[key]["cell_edge"],
                edgecolor='k', c=color, s=30, alpha=0.5, label=key)
    
plt.legend()
plt.xlabel("cell area [pxl^2]")
plt.ylabel("cell edge length [pxl]")
plt.show()